# Practise exercise

In this assignment we wish to use neural networks to estimate the two dimensional distribution of a dependent variable $Y = (Y_1,Y_2)$ as a function of covariates $x_1$ and $x_2$. This is *not* the hand-in exercise. Various questions and tasks are given throughout the notebook.

Before you start, make sure that the relevant packages are installed (for instance by using pip install). All the necessary packages can be found in the requirements.txt file.

In [ ]:
import tensorflow as tf
import numpy as np
import pyreadr
import matplotlib.pyplot as plt
from keras.layers import Dense, Input
from tensorflow.keras.models import Model

## Preliminaries

We first load, split, and standardize the data. The blocks below load the NeuralNetworks.Rdata file. The first 400 points are used as training set and the final 100 as test set. We have also created standardized versions of covariates and targets.

The blocks below load and split the data

In [ ]:
data = pyreadr.read_r('NeuralNetworks.Rdata')

In [ ]:
X = np.hstack((data['x1'], data['x2']))

In [ ]:
Y = np.hstack((data['Y1'], data['Y2']))

In [ ]:
X_train = X[0:400]
X_test = X[400:]
Y_train = Y[0:400]
Y_test = Y[400:]

The next block standardizes the covariates and targets.

Question: Why do we use X_mean and X_std in line 6 and not X_test and X_std?

In [ ]:
X_mean = np.mean(X_train, axis=0)
X_std = np.std(X_train, axis=0)
Y_mean = np.mean(Y_train, axis=0)
Y_std = np.std(Y_train, axis=0)

X_train_n = (X_train - X_mean) / X_std
X_test_n = (X_test - X_mean) / X_std

Y_train_n = (Y_train - Y_mean) / Y_std
Y_test_n = (Y_test - Y_mean) / Y_std

## Question (a)

Fit a neural network with four (dense) layers: three layers with 30 nodes and one output layer with two nodes. Use a ReLu activation function for the first three layers, and a linear for the last one. Use the mean squared error "mse" as loss function. Determine the mean squared error on the test data for $Y_1$ and $Y_2$ separately.

In [ ]:
def get_model_dual_output():
    """Create and compile a simple neural network with two outputs"""
    inputs = Input(shape=(2,))
    x = Dense(30, activation='relu')(inputs)
    x = Dense(30, activation='relu')(x)
    x = Dense(30, activation='relu')(x)
    outputs = Dense(2, activation='linear')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

Use the validation_split option to find the optimal number of epochs. Train on the entire training set for the eventual model (set validation_split to 0.0). We want to find the point where the validation loss no longer decreases (why?).

In [ ]:
model = get_model_dual_output()  # Get the model
model.fit(X_train_n, Y_train, epochs=..., batch_size=50, verbose=1, validation_split=0.2)  # Train the model

In [ ]:
predictions_single_model = model.predict(X_test_n, verbose=0)  # Get the predictions

In [ ]:
print(f'MSE model with dual output: {(np.mean(np.square(predictions_single_model - Y_test), axis=0))}')

## Question (b)

Now fit a similar neural network, but only with one output, with $Y_1$ as target variable. Compare the mean squared error on the test data for $Y_1$ with the model from (a). Is there any difference? If so, can you give an explanation for the difference?

Finish this code:

In [5]:
def get_model_single_ouput():
    """Create and compile a simple network with a one-dimensional output"""
    inputs = Input(shape=(2,))
    x = Dense(30, activation='relu')(inputs)
    x = Dense(30, activation='relu')(x)
    x = Dense(30, activation='relu')(x)
    outputs = ...
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return ...

Again, use the validation_split option to find the optimal training time.

In [ ]:
model_2 = get_model_single_ouput()
model_2.fit(X_train_n, Y_train[:, 0], epochs=..., batch_size=50, validation_split=0.0, verbose=1)

In [ ]:
print(f'MSE model single output: {}')

## Question (c)

We want to see if the model performs better with standardized targets as well. Use Y_train_n (the standardized version). Retrain the models from questions (a) and (b). Make sure to find the new optimal training time.

In [ ]:
model_3 = get_model_dual_output()
model_3.fit(X_train_n, Y_train_n, validation_split=0.0, epochs=..., batch_size=50, verbose=1)

In [ ]:
model_4 = get_model_single_ouput()
model_4.fit(X_train_n, Y_train_n[:, 0], validation_split=0.0, epochs=..., batch_size=50, verbose=1)

How does the optimal training time differ? Can you explain this? What would happen if you used the same training time as for the model with the original targets?

Transform the predictions back to the original scale in order to compare the results

In [ ]:
residuals_model_3 = model_3.predict(X_test_n) * ... + ... - ... # transform back to original scale
residuals_model_4 = model_4.predict(X_test_n) * ... + ... - ...

In [ ]:
print((np.mean(np.square(residuals_model_3), axis=0)))
print((np.mean(np.square(residuals_model_4), axis=0)))

## Question (d)


We now have a predictor for (the standardized) $(Y_1,Y_2)$. We view this predictor as an estimator for the regression function $f$, where we model $Y = (Y_1,Y_2)$ for given covariates $x$ as $Y \sim f(x)+ \mathcal{N}(0, \Sigma(x))$.
Here, $f(x) \in \mathbb{R}^{2}$. We also wish to estimate the covariance matrix $\Sigma(x)$.

The loss function can be used to train a network that predicts a covariance  metric. As targets, the networks gets the residuals of the first network (that outputs predictions $\hat{f}(x)$). This function outputs the negative loglikelihood of the data given a predicted covariance matrix. By using this as a loss function, we can find the optimal covariance matrix.

Answer the following questions about this loss function:

1. y_pred has three dimensions. The first two are the variances on logarithmic scale. These are then transformed to the normal scale in line 5. Why do we not output the variances directly?
2. How do we determine the off-diagonal term of the covariance matrix? Hint: see lines 6 through 8. Why is this done in this way? Why do we not directly output $cov(Y_1, Y_2)$?
3. In line 6, what is rho and why do we use this specific transformation?

In [ ]:
def covariance_loss(y_true, y_pred):
    residuals = y_true[:, 0:2]
    residuals = tf.expand_dims(residuals, axis=-1)
    log_variances = y_pred[:, 0:2]
    variances = tf.math.exp(log_variances) + 1e-6
    rho = 2 * (tf.math.sigmoid(y_pred[:, 2]) - 0.5)
    det = variances[:, 0] * variances[:, 1] * (1 - tf.square(rho))
    inv_cov_matrix =  tf.stack([
                                tf.stack([variances[:, 1] / det,
                                          -rho * tf.sqrt(variances[:, 0] * variances[:, 1]) / det], axis=1),
                                tf.stack([-rho * tf.sqrt(variances[:, 0] * variances[:, 1]) / det,
                                          variances[:, 0] / det], axis=1)
                               ], axis=1)
    mahalanobis_dist = tf.transpose(residuals, perm=[0, 2, 1])  @ inv_cov_matrix @ residuals
    log_likelihood = - 0.5 * tf.math.log(det)  - 0.5 * mahalanobis_dist
    return - log_likelihood

The model below will be used to obtain the covariance function. Finish the final layer. Be careful to select the correct number of outputs and the correct activation function. Note that in line 8, we have given our custom loss function.

In [ ]:
def get_covariance_model():
    inputs = Input(shape=(2))
    x = Dense(30, activation='relu')(inputs)
    x = Dense(30, activation='relu')(x)
    x = Dense(30, activation='relu')(x)
    outputs = Dense(..., activation=...)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss=covariance_loss)
    return model

Create and train the model. Find the optimal training time.

In [ ]:
covariance_model = get_covariance_model()
train_residuals = model_3.predict(X_train_n) - Y_train_n
covariance_model.fit(X_train_n, train_residuals, epochs=.., batch_size=50, validation_split=0.2)

We must transform the output of the model to a covariance matrix. Complete the code blow. Look at the custom loss function to see what parametrizations are used.

In [3]:
def sigmoid(x):
    """A simple sigmoid function"""
    return 1 / (1 + np.exp(-x))

In [ ]:
covariance_predictions = covariance_model.predict(X_test_n)
covariance_matrix = np.zeros((len(X_test_n), 2,2))
covariance_matrix[:, 0, 0] = ...(covariance_predictions[:, 0]) + ...
covariance_matrix[:, 1, 1] = ...(covariance_predictions[:, 1]) + ...
rho = (sigmoid(covariance_predictions[:, 1]) - 0.5) * ...
covariance_matrix[:, 0, 1] = covariance_matrix[:, 1, 0] = np.sqrt(covariance_matrix[:, 0, 0] * covariance_matrix[:, 1, 1]) * ...


Check the following matrices to see if everything works as expected.

In [ ]:
for i in range(5):
    print(covariance_matrix[i])

Can you think of a way to evaluate the performance of this estimator on a test set?

## Question (e)

We have a model that estimates $f(x)$ and a model that estimates $\Sigma(x)$. We can use these to simulate new targets. This allows us to repeat the entire procedure many times to see if this procedure produces sensible results. It is up to you to think of a way to quantify 'sensible results'. We first provide some code to simulate new targets.

The function below simulates a new set of targets given mean predictions and covariance matrix predictions.

In [4]:
def gen_new_targets(means, covariance_matrices):
    """Simulate new targets

    Arguments:
        means: An array containing the predicted means
        covariance_matrices: An array containing the predicted covariance matrices

    Returns:
        new_targets: An array containing simulated targets.
    """
    simulated_targets = np.zeros_like(means)
    for i, m in enumerate(means):
        new_target = np.random.multivariate_normal(m, cov=covariance_matrices[i])
        simulated_targets[i] = new_target
    return simulated_targets

Get the mean predictions and covariance matrix predictions for the training set. Make sure to use the same model for $\hat{f}(x)$ that was used to obtain the residuals that were used to obtain $\hat{\Sigma}(x)$.

In [ ]:
mean_predictions_train = model_3.predict(X_train_n, verbose=0)
covariance_predictions_train = covariance_model.predict(X_train_n, verbose=0)
covariance_matrices_train = np.zeros((len(X_train_n), 2,2))
covariance_matrices_train[:, 0, 0] = ...
covariance_matrices_train[:, 1, 1] = ...
rho = ...
covariance_matrices_train[:, 0, 1] = covariance_matrices_train[:, 1, 0] = ...

In [ ]:
new_targets = gen_new_targets(mean_predictions_train, covariance_matrices_train)

Plot a new set of simulated targets and compare with the original. Does it look reasonable?

In [ ]:
plt.figure(2)
plt.subplot(1, 2, 1)
plt.title('Simulated targets (normalized scale)')
plt.scatter(X_train[:, 1], new_targets[:, 0])
plt.scatter(X_train[:, 1], new_targets[:, 1])
plt.subplot(1, 2, 2)
plt.title('Original targets (normalized scale)')
plt.scatter(X_train[:, 1], Y_train_n[:, 0])
plt.scatter(X_train[:, 1], Y_train_n[:, 1])
plt.tight_layout()
plt.show()

To do: construct and run the simulation.

In [ ]:
N_simulations = 100
for i in range(N_simulations):
    # Simulate new targets
    # Train models
    # Evaluate models